# EDA on data from gas benchmarks

#### Maria Silva, December 2025

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [3]:
# plotting theme
sns.set_theme(
    style="whitegrid", palette="Set2", rc={"figure.dpi": 500, "axes.titlesize": 15}
)

## Load and process data

In [4]:
# Main directories
current_path = os.getcwd()
repo_dir = os.path.abspath(os.path.join(current_path, ".."))
data_dir = os.path.join(repo_dir, "data")

In this analysis, we are using data generated by running the [EEST benchmark suite](https://github.com/ethereum/execution-spec-tests/tree/main/tests/benchmark) with the [Nethermind benchmarking tooling](https://github.com/NethermindEth/gas-benchmarks). We extracted this data in 15-12-2025.

In [25]:
df = pd.read_csv(os.path.join(data_dir, "min_mgas_s_by_test_and_client_2025-12-15.csv"))
df = df.melt(id_vars=["test_title"], var_name="client", value_name="mgas/s")

# Fix rows with "k" and sort
df["mgas/s"] = df["mgas/s"].replace({" K": "e3"}, regex=True).astype(float)
df = df.sort_values(by="mgas/s")

# Parse test title
df["test_file"] = (
    df["test_title"].str.replace("tests_benchmark_", "").str.split(".py").str[0]
)
df["test_name"] = df["test_title"].str.split(".py__").str[1].str.split("[").str[0]
df["test_params"] = (
    df["test_title"]
    .str.split("[")
    .str[1]
    .str.split("]")
    .str[0]
    .str.split("engine_x")
    .str[1]
    .str[1:]
)
df["test_fork"] = df["test_title"].str.split("fork_").str[1].str.split("-").str[0]
# Parse opcodes
df["test_opcode"] = df["test_params"].str.extract(r"(?:opcode_|op_)([^-]+)")
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_modexp", "MODEXP", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"].str.contains("selfdestruct"), "SELFDESTRUCT", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_calldatacopy", "CALLDATACOPY", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_mcopy", "MCOPY", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_codecopy", "CODECOPY", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_returndatacopy", "RETURNDATACOPY", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"].str.contains("returndatasize"), "RETURNDATASIZE", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"].str.contains("extcodecopy"), "EXTCODECOPY", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_calldataload", "CALLDATALOAD", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_keccak", "KECCAK", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_shifts",
    df["test_params"].str[-3:],
    df["test_opcode"],
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_selfbalance", "SELFBALANCE", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_msize", "MSIZE", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_jumpdests", "JUMPDEST", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"].str.contains("jumpi"), "JUMPI", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_jumps", "JUMP", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_tstore", "TSTORE", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_tload", "TLOAD", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("SSTORE"), "SSTORE", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("SLOAD"), "SLOAD", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_log_opcodes", "LOG", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_calldatasize", "CALLDATASIZE", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_blobhash", "BLOBHASH", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_blockhash", "BLOCKHASH", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_callvalue", "CALLVALUE", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bn128_add"), "ecAdd", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bn128_mul"), "ecMul", df["test_opcode"]
)
df["test_opcode"] = np.where(
    (df["test_params"].str.contains("bn128"))
    & (df["test_params"].str.contains("pairing")),
    "ecPairing",
    df["test_opcode"],
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_amortized_bn128_pairings", "ecPairing", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("ec_pairing"), "ecPairing", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("point_evaluation"),
    "point evaluation",
    df["test_opcode"],
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("blake2f"), "blake2f", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("ecrecover"), "ecRecover", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("SHA2-256"), "SHA2-256", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("RIPEMD-160"), "RIPEMD-160", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("IDENTITY"), "identity", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bls12_g1add"), "BLS12_G1ADD", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bls12_g1msm"), "BLS12_G1MSM", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bls12_g2add"), "BLS12_G2ADD", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bls12_g2msm"), "BLS12_G2MSM", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bls12_pairing_check"),
    "BLS12_PAIRING_CHECK",
    df["test_opcode"],
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bls12_fp_to_g1"),
    "BLS12_MAP_FP_TO_G1",
    df["test_opcode"],
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bls12_fp_to_g2"),
    "BLS12_MAP_FP2_TO_G2",
    df["test_opcode"],
)
df["test_opcode"] = np.where(
    df["test_name"].str.contains("jumpdest_analysis"),
    "JUMPDEST",
    df["test_opcode"],
)
# Drop Nas and unnecessary column
df = df.drop(columns=["test_title"])
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2540 entries, 2604 to 1840
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   client       2540 non-null   object 
 1   mgas/s       2540 non-null   float64
 2   test_file    2540 non-null   object 
 3   test_name    2540 non-null   object 
 4   test_params  2540 non-null   object 
 5   test_fork    2540 non-null   object 
 6   test_opcode  2540 non-null   object 
dtypes: float64(1), object(6)
memory usage: 158.8+ KB


In [27]:
worse_df = (
    df[df["mgas/s"]>0] # ignore zeros
    .groupby("test_opcode")
    ["mgas/s"]
    .min()
    .reset_index()
)
worse_df

,test_opcode,mgas/s
0,ADD,32.9
1,ADDMOD,80.1
2,ADDRESS,72.0
3,AND,85.5
4,BALANCE,234.0
...,...,...
153,ecMul,61.4
154,ecPairing,57.8
155,ecRecover,60.3
156,identity,83.7


## Underpriced opcodes

#### 100M gas limit -> 35M gas/second

In [30]:
worse_df[worse_df["mgas/s"]<=35].sort_values(by="mgas/s")

,test_opcode,mgas/s
129,STATICCALL,6.46
52,EXTCODESIZE,6.48
18,CALLCODE,6.54
17,CALL,6.56
30,DELEGATECALL,6.65
50,EXTCODECOPY,6.80
51,EXTCODEHASH,9.35
67,MODEXP,10.80
117,SDIV,24.10
116,SAR,30.00
